In [94]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_percentage_error
from itertools import product
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')


for item in ['APU0000706111']:
    
    target=item
    path = f'/Users/angelobenedicto/Documents/Commodities Hedger/Commodities/{target}.csv'

    # Load the dataset
    data = pd.read_csv(path, parse_dates=['DATE'], index_col='DATE')
    data[target] = pd.to_numeric(data[target], errors='coerce')
    data = data['2014':'2023-02-01']
    data.interpolate(method='linear', inplace=True)

    # Define the p, d, q, and seasonal P, D, Q, S
    p = d = q = range(0, 3)
    seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(product(p, d, q))]

    # Temporal Cross Validation sets
    n_periods = [6, 9, 12]

    # DataFrame to store results
    results = pd.DataFrame(columns=['pdq', 'seasonal_pdq', 'hold_out', 'MAPE'])

    total_iterations = len(list(product(p, d, q))) * len(seasonal_pdq) * len(n_periods)

    # Initialize progress bar
    with tqdm(total=total_iterations, desc="Progress", unit="iteration") as pbar:
        for pdq in product(p, d, q):
            for seasonal in seasonal_pdq:
                for hold in n_periods:
                    try:
                        # Splitting the data
                        train = data[:-hold]
                        test = data[-hold:]

                        # Fit SARIMA Model
                        model = SARIMAX(train, order=pdq, seasonal_order=seasonal, enforce_stationarity=False, enforce_invertibility=False)
                        model_fit = model.fit(disp=False)

                        # Forecast
                        predictions = model_fit.get_forecast(steps=hold)
                        prediction_values = predictions.predicted_mean

                        # Calculate MAPE
                        mape = mean_absolute_percentage_error(test, prediction_values)

                        # Store results
                        results = pd.concat([results, pd.DataFrame({
                            'pdq': [pdq],
                            'seasonal_pdq': [seasonal],
                            'hold_out': [hold],
                            'MAPE': [mape]
                        })], ignore_index=True)

                    except Exception as e:
                        print(f'Error: {e} with PDQ {pdq}, Seasonal PDQ {seasonal}, Hold-Out {hold}')

                    # Update progress bar
                    pbar.update(1)

    #Pivot Results
    pivot_df = results.pivot_table(index=['pdq', 'seasonal_pdq'], columns='hold_out', values='MAPE', aggfunc='first')
    pivot_df.reset_index(inplace=True)
    pivot_df['MEAN MAPE'] = pivot_df[[6, 9, 12]].mean(axis=1)
    pivot_df.sort_values(by='MEAN MAPE')
    pivot_df.to_csv(f'{target}_params_list.csv')

    #Get Best Params
    best_params = pivot_df.sort_values(by='MEAN MAPE').iloc[0]
    best_pdq = best_params['pdq']
    best_seasonal_pdq = best_params['seasonal_pdq']

    # Fit model using best parameters on entire data set
    final_model = SARIMAX(data, order=best_pdq, seasonal_order=best_seasonal_pdq, enforce_stationarity=False, enforce_invertibility=False)
    final_model_fit = final_model.fit(disp=False)

    # Forecast the next 12 steps
    forecast = final_model_fit.get_forecast(steps=12)
    forecast_values = forecast.predicted_mean
    forecast_values.to_csv(f'{target}_forecast.csv')

Progress: 100%|██████████████████████| 2187/2187 [24:45<00:00,  1.47iteration/s]


In [97]:
#Load Comm Lookup
comm_lookup = pd.read_excel('/Users/angelobenedicto/Documents/Commodities Hedger/Commodities/CommLookUp.xlsx')

#Farm Forecasts
forecast_df = pd.DataFrame(columns=['DATE', 'FORECAST', 'FRED Ticker'])

#Combine DataFrames
for item in ['APU0000706111', 'PSMEAUSDM','APU000072610','CES2000000008']:
    comm_df = pd.read_csv(f'/Users/angelobenedicto/Documents/GitHub/BarnGPT/Forecasts/{item}_forecast.csv')
    comm_df = pd.DataFrame(comm_df)
    comm_df['COMM'] = item
    comm_df.columns=['DATE', 'FORECAST', 'FRED Ticker']
    forecast_df = pd.concat([forecast_df, comm_df], axis=0)
                          
forecast_df = forecast_df.merge(comm_lookup[['FRED Ticker', 'Code']], how='left', on='FRED Ticker')

In [90]:
#Farm Actuals
actuals_df = pd.DataFrame(columns=['DATE', 'ACTUALS', 'FRED Ticker'])

#Combine DataFrames
for item in ['APU0000706111', 'PSMEAUSDM','APU000072610','CES2000000008']:
    comm_df = pd.read_csv(f'/Users/angelobenedicto/Documents/GitHub/BarnGPT/Commodities/{item}.csv')
    comm_df = pd.DataFrame(comm_df)
    comm_df['COMM'] = item
    comm_df.columns=['DATE', 'ACTUALS', 'FRED Ticker']
    actuals_df = pd.concat([actuals_df, comm_df], axis=0)

actuals_df = actuals_df.merge(comm_lookup[['FRED Ticker', 'Code']], how='left', on='FRED Ticker')

In [99]:
#Export Forecasts and Actuals to Excel
forecast_df.to_excel('forecast.xlsx')
actuals_df.to_excel('actuals.xlsx')